In [44]:
import numpy as np
import random
import csv

data = []

# Betöltés
# kódolási séma specifikálása
with open('data.csv','r', encoding='utf-8-sig') as f:
    csvreader = csv.DictReader(f)
    for item in csvreader:
        data.append([ item['DATE'], item['AUTHOR'], item['CONTENT'], item['CLASS'] ])

# A 'data' tömb elemei: ['dátum string', 'szerző', 'komment', 'osztály cimke ('0': nem spam, '1': spam)']
        
# Train/test szétválasztás
split = 0.7
data = np.asarray(data)
perm = np.random.permutation(len(data))

train = data[perm][0:int(len(data)*split)]
test = data[perm][int(len(data)*split):]

print('Train set: ', np.shape(train))
print('Test set: ', np.shape(test))

# Buta osztályozó
def dumb_classify(data):
    threshold = 0.3
    if random.random() > threshold:
        return '1'
    else:
        return '0'


# Használd a 'train' adatokat az osztályozó módszer kidolgozására, a 'test' adatokat kiértékelésére!
# Lehetőleg használj gépi tanulást!
# Dokumentáld az érdekesnek tartott kísérleteket is!

# Példa kiértékelés 'recall' számításával. 
# Kérdés: Milyen egyéb metrikát használnál kiértékelésre és miért? 
sum_positive = 0
found_positive = 0

for datapoint in test:
    if datapoint[-1] == '1':
        sum_positive += 1
        if dumb_classify(datapoint) == '1':
            found_positive += 1
    
print('Recall:', found_positive / sum_positive)

Train set:  (1367, 4)
Test set:  (586, 4)
Recall: 0.6917562724014337


## Importálandó modulok

In [45]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import emoji
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC

tokenizer = TweetTokenizer()
stemmer = EnglishStemmer()
stp_wrds = set(stopwords.words('english') + [''])

## Előfeldolgozás

In [46]:
train_df = pd.DataFrame(train, columns=["date", "author", "text", "spam"])
test_df = pd.DataFrame(test, columns=["date", "author", "text", "spam"])

In [47]:
x = train_df['text'].iloc[2]
tokenizer.tokenize(x.lower())

['want',
 'to',
 'win',
 'borderlands',
 'the',
 'pre',
 '-',
 'sequel',
 '?',
 'check',
 'my',
 'channel',
 ':)',
 '\ufeff']

In [48]:
def process_word(word):
    # szamok helyettesitese
    if re.match(r'\d+', word) is not None:
        return '$'
    # emoji es smiley helyettesitese
    elif re.match(r'(?::|;|=)(?:-)?(?:\)|D|P|\()', word) is not None or word in emoji.UNICODE_EMOJI:
        return 'EMOJI'
    # url linkek helyettesitese
    elif re.match(r'((http|https):\/\/)?([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])?', word) is not None:
        return 'URL'
    else:
        # irasjelek eltavolitasa kiveve felkialtojel
        out = re.sub(r'[^A-Za-z0-9!]', '', word)
        # ha egy betu egymas utan tobb, mint 3szor szerepel egy szoban, akkor valoszinuleg emfatikus a hasznalata
        # pl. woooowwwwwwww
        # ezeket egyetlen beture cserelem
        out = re.sub(r'(.)\1+', r'\1', out)
        # stemmelés
        return stemmer.stem(out)
    
def preprocessor(comment_text):
    # \ufeff karakterek eltűntetése
    clean_text = comment_text.replace('\ufeff', ' ')
    # html kodok eltuntetese
    clean_text = re.sub(r'<br\s*\/*>', ' ', clean_text)
    # kisbetűsítés és tokenizálás
    token_list = tokenizer.tokenize(clean_text.lower())
    # elofeldolgozes, tagekre csereles, stemmeles
    p_token_list = [process_word(word) for word in token_list]
    # stop word eltavolitas
    return [item for item in p_token_list if item not in stp_wrds]

In [49]:
print('Egy példa a preprocesszor működéséről')
print(x)
print(preprocessor(x))
preprocessor('HAHAA THIS DANCE IS TIGHTTTT<br /><br />I know :) !!!"^*&^?')

Egy példa a preprocesszor működéséről
want to win borderlands the pre-sequel? check my channel :)﻿
['want', 'win', 'borderland', 'pre', 'sequel', 'check', 'chanel', 'EMOJI']


['haha', 'danc', 'tight', 'know', 'EMOJI', '!', '!', '!']

## Vektorizálás

In [50]:
# vektorizalas eloszor count vektorizatorral
"""vectorizer = CountVectorizer(lowercase=False,
                             ngram_range=(1, 1), 
                             analyzer=preprocessor, 
                             max_df=0.95, 
                             min_df=5, 
                             max_features=None)
"""
vectorizer = TfidfVectorizer(lowercase=False,
                            ngram_range=(1,1),
                            analyzer=preprocessor,
                            max_df=0.95,
                            min_df=5)
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['spam'].map(lambda x: int(x))
X_test = vectorizer.transform(test_df['text'])
y_test = test_df['spam'].map(lambda x: int(x))

## Feature engineering - extra features

In [51]:
#spams = train_df[train_df['spam'] == '1']
#print(spams['text'].head(25))
#train_df['date'].head()

In [52]:
def calc_char_ratios(comment):
    if len(comment) < 1:
        return (0, 0, 0)
    # betűk aránya
    lett_ratio = len(re.findall(r'[a-zA-Z]', comment))/len(comment)
    # nagybetűk aránya
    upper_ratio = len(re.findall(r'[A-Z]', comment))/len(comment)
    # irasjelek aránya
    punc_ratio = len(re.findall(r'[^a-zA-Z0-9\s]', comment))/len(comment)
    return (lett_ratio, upper_ratio, punc_ratio)
    
def deter_posting_time(date):
    # posztolás ideje: 0 = NaN, 1 = hajnal, 2 = delelott, 3 = delutan, 4 = este 
    if len(date) < 1:
        return 0
    hour = int(re.search(r'T(\d{2}):', date).groups()[0])
    if hour < 6:
        return 1
    elif hour < 12:
        return 2
    elif hour < 18:
        return 3
    else:
        return 4

In [53]:
# extra feature-ok hozzaadasa a tanitoadatokhoz
n = X_train.shape[1]
feature_names = ['lett_ratio', 'upper_ratio', 'punc_ratio', 'posting_time']
extra_train = train_df['text'].map(lambda x: calc_char_ratios(x))
extra_train = extra_train.apply(pd.Series)
extra_train[3] = train_df['date'].map(lambda x: deter_posting_time(x))
X_train = hstack((X_train, extra_train))

#update vecotorizer vocabulary
for i in range(4):
    vectorizer.vocabulary_[feature_names[i]] = n + i

# extra feature-ok hozzaadasa a tesztadatokhoz
extra_test = test_df['text'].map(lambda x: calc_char_ratios(x))
extra_test = extra_test.apply(pd.Series)
extra_test[3] = test_df['date'].map(lambda x: deter_posting_time(x))
X_test = hstack((X_test, extra_test))

print(X_train.shape, X_test.shape)

(1367, 393) (586, 393)


## Multinomial Naive Bayes classifier

A spam detektálás egy klasszifikációs feladat, amelyet az irodolom szerint a Naív Bayes osztályozók kifejezetten jól képesek kezelni. A Naív Bayes-féle osztályozók továbbá kellően robosztusak, gyorsak és kevés tanulóadattal is jól működnek, így baseline modellként érdemes ezzel kezdeni.

Mivel a nyelvi adatok feldolgozásához gyakorisági alapú vektorizálót használtam, így a naív Bayes osztályozók multinomiális változatát választottam.

In [54]:
multi_nb = MultinomialNB()
multi_nb.fit(X_train, y_train)
print('Recall a tanulóadatokon')
print(classification_report(y_train, multi_nb.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, multi_nb.predict(X_test)))
# performance logs
# CountVectorizer max_df 1.0, min_df 1, MN Naive Bayes (alpha 1), test set: 0 recall 0.84, 1 recall 0.95
# CountVectorizer max_df 1.0, min_df 1, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.89, 1 recall 0.95
# CountVectorizer max_df 0.95, min_df 5, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.96, 1 recall 0.88
# TfidfVectorizer max_df 1.0, min_df 1, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.95, 1 recall 0.91
# TfidfVectorizer max_df 0.95, min_df 5, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.93, 1 recall 0.94

Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.92      0.96      0.94       643
          1       0.96      0.93      0.95       724

avg / total       0.94      0.94      0.94      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.94      0.93      0.94       307
          1       0.93      0.94      0.93       279

avg / total       0.93      0.93      0.93       586



In [56]:
# alpha állítása
mn_nb = MultinomialNB()
params = {'alpha': [0.01, 0.1, 1, 2, 5, 10]}
grid_CV = GridSearchCV(mn_nb, params, cv=10, refit=True)
grid_CV.fit(X_train, y_train)
print('Best alpha: {}'.format(grid_CV.best_params_))
print('Recall a tanulóadatokon')
print(classification_report(y_train, grid_CV.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, grid_CV.predict(X_test)))
# best model with TfidfVectorizer max_df 0.95, min_df 5, with extra featerus
# MN_Bayes alpha 2, 0 recall on test dataset 0.94; 1 recall on test dataset 0.94

Best alpha: {'alpha': 2}
Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.92      0.96      0.94       643
          1       0.96      0.93      0.94       724

avg / total       0.94      0.94      0.94      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.94      0.94      0.94       307
          1       0.94      0.94      0.94       279

avg / total       0.94      0.94      0.94       586



## SVM

A Support Vector Machine algoritmusok ugyancsak jól szoktak szerepelni a szövegkategorizációs feladatokban, így a következő részben megnézem, hogy milyen pontossággal képesek a kommentek közül kiválogatni a spamokat. A Naív Bayes-féle modellek eddigi legjobbjai 94%-os recall körül teljesítettek.

In [57]:
svc_model = SVC()
svc_params = {'C': [0.01, 0.1, 1, 10],
             'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
             'degree': [2],
             'class_weight': ['balanced']}
svc_grid = GridSearchCV(svc_model, svc_params)
svc_grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.1, 1, 10], 'degree': [2], 'class_weight': ['balanced'], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [60]:
print('Best params: {}'.format(svc_grid.best_params_))
print('Recall a tanulóadatokon')
print(classification_report(y_train, svc_grid.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, svc_grid.predict(X_test)))

Best params: {'C': 10, 'degree': 2, 'class_weight': 'balanced', 'kernel': 'linear'}
Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.98      0.99      0.99       643
          1       0.99      0.98      0.99       724

avg / total       0.99      0.99      0.99      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.95      0.95      0.95       307
          1       0.95      0.95      0.95       279

avg / total       0.95      0.95      0.95       586



Az SVM klasszifikátor 1%-kal teljesít jobban, mint a Naív Bayes modell. Átlagosan a recall 95% a tesztadatokon. Mivel a legjobb eredményt a lineáris kernel érte el, így érdemes lehel logaritmikus regresszióval is ellenőrizni az eredményeket, hiszen a scikit-learn Logistic Regression moduljában több 

In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, multi_nb.predict(X_test))

array([[250,  47],
       [ 14, 275]])

In [42]:
# todo
# SVM, random forest

## Lehetséges változók

1. Url link; kulcsszabak *check out, visit, subscribe, follow me, money*
2. Nagybetűk, írásjelek pl. felkiáltójelek aránya
3. Felhasználó (csak 21 komment van ugyanazon felhasználótól a spamok között) ``np.unique(spams[:, 1])``
4. Posztolás ideje

## Modellek építése

## Lehetséges problémak

### Osztályok közötti aránytalanság

In [ ]:
spams = train[np.where(train == '1')[0]]
len(train)/len(spams)

Majdnem 2-szer annyi a negatív példa, mint pozitív.

In [ ]:
# egyedi felhasznalonevek szama
len(spams) - len(np.unique(spams[:, 1]))